In [1]:
"""Demo notebook."""

import os
from niceview.utils.raster import geo_ref_raster
from niceview.utils.mask import sparse_npz_to_array, mask_overlay_image
from niceview.utils.tools import txt_to_list, select_col_from_name, normalize_nonzero_to_range
from niceview.utils.cell import get_nuclei_pixels, paint_regions
import pandas as pd
import scipy

In [2]:
# configurations
DATA_PATH = '../examples/data/'

In [3]:
# args
file = {
    'cells-gene-names': 'gt-iz-p9-rep2-cells-gene-names.txt',
    'cells-gene': 'gt-iz-p9-rep2-cells-gene.npz',
    'cells-info': 'gt-iz-p9-rep2-cells-info.csv',
    'mask': 'gt-iz-p9-rep2-mask.npz',
}

mask_filtered_relabeled_name = 'gt-iz-p9-rep2-mask-filtered-relabeled.npz'  # variable
img_name = 'gt-iz-p9-rep2-img.tiff'  # variable
img_gis_name = 'gt-iz-p9-rep2-img-gis.tiff'  # variable
img_masked_name = 'gt-iz-p9-rep2-img-masked.png'  # variable
img_masked_gis_name = 'gt-iz-p9-rep2-img-masked-gis.tiff'  # variable
cell_info_name = 'gt-iz-p9-rep2-cells-info.csv'  # variable

selected_gene_id = 'ENSG00000065534'

In [4]:
# read data
mask = sparse_npz_to_array(os.path.join(DATA_PATH, 'gt-iz-p9-rep2-mask.npz'))
cell_pos = pd.read_csv(DATA_PATH + 'gt-iz-p9-rep2-cells-info.csv')[['x', 'y']].values
cells_gene = sparse_npz_to_array(os.path.join(DATA_PATH, 'gt-iz-p9-rep2-cells-gene.npz'))
gene_names = txt_to_list(os.path.join(DATA_PATH, 'gt-iz-p9-rep2-cells-gene-names.txt'))

# get nuclei pixels
gene = select_col_from_name(cells_gene, gene_names, selected_gene_id)
matched_regions = get_nuclei_pixels(mask, cell_pos)

# relabel mask
mask_filtered_relabeled = paint_regions(mask.shape, matched_regions, cell_colors_list=gene + 1e-6)
mask_filtered_relabeled = scipy.sparse.csr_matrix(mask_filtered_relabeled.data)

# save relabeled mask
scipy.sparse.save_npz(DATA_PATH + 'gt-iz-p9-rep2-mask-filtered-relabeled.npz', mask_filtered_relabeled)

# read relabeled mask
mask_filtered_relabeled = sparse_npz_to_array(os.path.join(DATA_PATH, mask_filtered_relabeled_name))

# normalize relabeled mask
mask_filtered_relabeled_norm = normalize_nonzero_to_range(mask_filtered_relabeled)

In [65]:
# image gis
img_gis_path = geo_ref_raster(
    os.path.join(DATA_PATH, img_name),
    os.path.join(DATA_PATH, img_gis_name),
    overwrite=False,
)

# image with mask overlay witout geo-referencing
img_masked_path = mask_overlay_image(
    os.path.join(DATA_PATH, img_name),
    mask_filtered_relabeled_norm,
    os.path.join(DATA_PATH, img_masked_name),
    mask_opacity=0.5,
    colormap='jet',
    overwrite=False,
)

# image with mask overlay with geo-referencing
img_masked_gis_path = geo_ref_raster(
    img_masked_path,
    os.path.join(DATA_PATH, img_masked_gis_name),
    overwrite=False,
)

File ../examples/data/gt-iz-p9-rep2-img-gis.tiff already exists.
File ../examples/data/gt-iz-p9-rep2-img-masked.png already exists.
File ../examples/data/gt-iz-p9-rep2-img-masked-gis.tiff already exists.
